In [119]:
from ortools.linear_solver import pywraplp

In [120]:
solver = pywraplp.Solver.CreateSolver('CP_SAT')

In [121]:
material_list = [0,1]
day_list = [0, 1, 2]
machine_list = [0,1,2]
machine_mat_dict = {(0,0):150, (0,1):75, (1,0):50, (1,1):0, (2,0):0, (2,1):100}

In [122]:
machine_mat_date_dict = {}
for (i,j) in machine_mat_dict.keys():
    for k in day_list:
        machine_mat_date_dict[(i,j,k)] = machine_mat_dict[(i,j)]

In [123]:
demand_day_dict = {(0,0):40, (1,0):20, (0,1):10, (1,1): 36, (0,2):20, (1,2):50}

In [124]:
margin_dict = {0:300, 1:500}

In [125]:
x = {}
for i in machine_list:
    for j in material_list:
        for k in day_list:
            x[(i,j,k)] = solver.NumVar(0,100000000, 'x_%i_%i_%i' % (i,j,k))
            solver.Add(x[(i,j,k)] <= machine_mat_date_dict[(i,j,k)])

In [126]:
diff = {}
for j,k in demand_day_dict.keys():
    diff[(j,k)] = solver.NumVar(-10000000,1000000, 'diff_%i_%i' %(j,k))

In [127]:
# we are makin sure the difference wont cross 110% of demand
for j,k in demand_day_dict.keys():
    solver.Add(diff[(j,k)] == sum(x[(i,j,k)] for i in machine_list) - demand_day_dict[(j,k)])
    #solver.Add(diff[(j,k)] <= 1.1*demand_day_dict[(j,k)])

In [128]:
margin = {}
for j in margin_dict.keys():
    margin[j] = solver.NumVar(0,100000000, 'margin_%i'%j)

In [129]:
for j in margin_dict.keys():
    solver.Add(margin[j] == sum(x[(i,j,k)] for i in machine_list for k in day_list) * margin_dict[j])

In [130]:
solver.Maximize(sum(diff[(j,k)] for j,k in demand_day_dict.keys()) - sum(margin[j] for j in margin_dict.keys()))

In [131]:
solver.Solve() ==pywraplp.Solver.OPTIMAL

True

In [132]:
for i in machine_list:
    for j in material_list:
        for k in day_list:
            if x[(i,j,k)].solution_value():
                print("machine ",i, "material ", j, "day ", k,x[(i,j,k)].solution_value(), x[(i,j,k)].solution_value() - demand_day_dict[(j,k)], 1.1*demand_day_dict[(j,k)])